In [ ]:
import pdfplumber
import pandas as pd
import re

# Define the column names for the DataFrame
column_names = ["Puzzle Number", "Game", "Location & Year", "Solution: Desc Notation",
                "Solution: Modern Notation", "Processing Notes"]

# Create an empty DataFrame
df = pd.DataFrame(columns=column_names)

# Provide the path to your PDF file
pdf_path = '/path/to/your/file/chess-endgames.pdf'

# Output file path
output_file_path = '/path/to/your/output/file/endgames-py-output.txt'

In [ ]:

# Open the PDF file
with pdfplumber.open(pdf_path) as pdf:
    for i in range(785, 1153):  # Start from page 715, end at page 1008
        page = pdf.pages[i]
        text = page.extract_text()

        # Initialize puzzle_number to "Unknown"
        puzzle_number = "Unknown"
        try:
            # Extract the puzzle number, game, location & year, and solution
            puzzle_number = re.findall(r'(\d+)', text)
            print(f'Puzzle Number: {puzzle_number}')
            puzzle_number = puzzle_number[0]

            game = re.findall(r'([A-Za-z -]+)', text)
            print(f'Game: {game}')
            game = game[1]

            location_year = re.findall(r'([A-Za-z ,0-9]+)', text)
            print(f'Location & Year: {location_year}')
            location_year = location_year[2]

            solution = re.findall(r'((?:[♚♛♜♝♞♟♔♕♖♗♘♙].*? )+\d-\d)', text)
            print(f'Solution: {solution}')
            solution = solution[0]

            # Add a row to the DataFrame
            df = df.append({"Puzzle Number": puzzle_number, "Game": game, "Location & Year": location_year,
                            "Solution: Desc Notation": solution, "Processing Notes": ""}, ignore_index=True)

            # Print an update for every 400 puzzles added to the DataFrame
            if (i - 714 + 1) % 400 == 0:
                print(f'Processed {i - 785 + 1} puzzles.')
        except Exception as e:
            df = df.append({"Puzzle Number": puzzle_number, "Game": "", "Location & Year": "",
                            "Solution: Desc Notation": "", "Processing Notes": str(e)}, ignore_index=True)

# Save the DataFrame to a file
df
